# NVIDIA AI Blueprint for Video Search and Summarization: 4xL40S Hybrid Deployment

This notebook deploys the NVIDIA AI Blueprint for Video Search and Summarization using a **HYBRID APPROACH** with 4xL40S GPUs:
- **GPU 0**: Reranker (Local)
- **GPU 1**: Embedding (Local) 
- **GPU 2,3**: VLM - Vision Language Model (Local)
- **Remote API**: LLM (NVIDIA Hosted)

This setup provides **50% cost savings** compared to the 8xL40S setup while maintaining excellent performance.

**Note**: This notebook is optimized for **4xL40S on CRUSOE Cloud Provider with Ephemeral storage**

## Prerequisites

### Obtain NVIDIA API Keys

This key will be used to pull relevant models and containers from build.nvidia.com and NGC, plus access remote LLM services.

Generate the key from [NGC Portal](https://ngc.nvidia.com/) using the same account used to apply for [Blueprint Early Access.](https://developer.nvidia.com/ai-blueprint-for-video-search-and-summarization-early-access/join)

Follow the instructions to [Generate NGC API Key.](https://docs.nvidia.com/ngc/gpu-cloud/ngc-user-guide/index.html#generating-api-key)

<div class="alert alert-block alert-success">
    <b>Note:</b>  If you have authentication issues when pulling the NIMs, please verify you have the following <a href="https://org.ngc.nvidia.com/subscriptions" target="_blank">Subscriptions</a>: <strong>NVIDIA Developer Program</strong>
 </div>



In [ ]:
import os

os.environ["NGC_API_KEY"] = "***" #Replace with your key

### Specify the Vision Language Model (VLM)

By default, we are using [VILA-1.5](https://build.nvidia.com/nvidia/vila) model. You could use other models like [NVILA](https://huggingface.co/Efficient-Large-Model/NVILA-15B), GPT-4o, etc.

In [ ]:
os.environ["VLM_MODEL_TO_USE"] = "vila-1.5" #or choose nvila
os.environ["MODEL_PATH"] = "ngc:nim/nvidia/vila-1.5-40b:vila-yi-34b-siglip-stage3_1003_video_v8" #for nvila, use "git:https://huggingface.co/Efficient-Large-Model/NVILA-15B"

Ensuring the user is on right path

In [ ]:
# !git clone https://github.com/NVIDIA-AI-Blueprints/video-search-and-summarization.git
# %cd video-search-and-summarization/docker/launchables

%cd ./docker/launchables
!ls

### Verify the Driver and CUDA version to be the following:
- Driver Version: 535.x.x
- CUDA Version 12.2

In [ ]:
!nvidia-smi

If the driver version doesn't match in the above step:
- Update the Driver to 535
- Reboot the system
- Set ```NGC_API_KEY``` again

In [ ]:
# !sudo apt install nvidia-driver-535 -y
# !sudo reboot now

## Deployment: 4xL40S Hybrid Configuration

We will be using a hybrid approach:
- **Local Models**: VLM, Reranker, Embedding
- **Remote API**: LLM (saves 4 GPUs!)

### GPU Configuration for 4xL40S

| Component | GPU Assignment | Model |
|-----------|---------------|---------|
| Reranker  | GPU 0        | llama-3.2-nv-rerankqa-1b-v2 |
| Embedding | GPU 1        | llama-3.2-nv-embedqa-1b-v2 |
| VLM       | GPU 2,3      | VILA-1.5 or NVILA |
| LLM       | Remote API   | llama-3.1-70b-instruct |

**Cost Savings: 50% compared to 8xL40S setup!**


### Step 1: Set Environment Variables and Login to Docker

In [7]:
#os.environ["LOCAL_NIM_CACHE"] = os.path.expanduser("~/.cache/nim") #default cache location
os.environ["LOCAL_NIM_CACHE"] = os.path.expanduser("/ephemeral/cache/nim") #updating with ephemeral storage
os.makedirs(os.environ["LOCAL_NIM_CACHE"], exist_ok=True)

In [ ]:
%%bash
echo "${NGC_API_KEY}" | docker login nvcr.io -u '$oauthtoken' --password-stdin

Updating the docker storage path to Ephemeral storage

In [ ]:
import json, subprocess, time

storage_path = "/ephemeral/cache/docker"

daemon_file = "/etc/docker/daemon.json" #update the path if required
config = {}
try:
    config = json.load(open(daemon_file)) if os.path.exists(daemon_file) else {}
except PermissionError:
    print("Cannot read the file. Try running with elevated privileges or check docker deamon file path.")

config["data-root"] = storage_path
config_str = json.dumps(config, indent=4)

subprocess.run(f"echo '{config_str}' | sudo tee {daemon_file} > /dev/null", shell=True, check=True)
subprocess.run("sudo systemctl restart docker", shell=True, check=True)

time.sleep(5)

# Verify new storage location
print(subprocess.run("docker info | grep 'Docker Root Dir'", shell=True, capture_output=True, text=True).stdout)

### Step 2: LLM NIM - SKIPPED (Using Remote API)

**🚀 COST OPTIMIZATION:** We're using remote NVIDIA API for LLM instead of local deployment.

**Benefits:**
- Saves 4 GPUs (was using GPUs 0,1,2,3)
- Reduces infrastructure costs by 50%
- No model download/setup time
- Always up-to-date model

The LLM API will be configured in the environment variables for the main blueprint container.

In [ ]:
print("✅ LLM NIM - SKIPPED")
print("🔄 Using remote NVIDIA API for LLM instead")
print("💰 Cost savings: 4 GPUs freed up!")
print("⚡ Faster deployment: No LLM model download needed")

### Step 3: Launch the Reranker NIM on GPU 0

**Modified for 4xL40S:** Using GPU 0 instead of GPU 4

In [ ]:
!docker run -it --rm \
    --gpus '"device=0"' \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 9235:8000 \
    -d \
    nvcr.io/nim/nvidia/llama-3.2-nv-rerankqa-1b-v2:latest

### Step 4: Launch the Embedding NIM on GPU 1

**Modified for 4xL40S:** Using GPU 1 instead of GPU 5

In [ ]:
!docker run -it --rm \
    --gpus '"device=1"' \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 9234:8000 \
    -d \
    nvcr.io/nim/nvidia/llama-3.2-nv-embedqa-1b-v2:latest

### Step 5: Verify Local NIMs are Running

After running the following cell, you should see **TWO containers** (Reranker and Embedding only).

**Note:** No LLM container since we're using remote API!

In [ ]:
!docker ps
print("\n🎯 Expected: 2 containers (Reranker on GPU 0, Embedding on GPU 1)")
print("📝 LLM container is NOT shown - we're using remote API!")

### Step 6: Prepare Environment for Blueprint with Hybrid Configuration

Before launching the blueprint, we need to configure it for:
- **VLM on GPUs 2,3** (local)
- **Remote LLM API** configuration

In [ ]:
# Update .env file to use GPUs 2,3 for VLM instead of 6,7
import re

# Read current .env file
with open('.env', 'r') as f:
    env_content = f.read()

# Update NVIDIA_VISIBLE_DEVICES for VLM to use GPUs 2,3
env_content = re.sub(
    r'NVIDIA_VISIBLE_DEVICES=.*',
    'NVIDIA_VISIBLE_DEVICES=2,3',
    env_content
)

# Add remote LLM configuration
remote_llm_config = '''
# Remote LLM Configuration (4xL40S Hybrid Setup)
USE_REMOTE_LLM=true
REMOTE_LLM_API_BASE=https://integrate.api.nvidia.com/v1
REMOTE_LLM_MODEL=meta/llama-3.1-70b-instruct
'''

# Add remote config if not already present
if 'USE_REMOTE_LLM' not in env_content:
    env_content += remote_llm_config

# Write back to .env
with open('.env', 'w') as f:
    f.write(env_content)

print("✅ Updated .env for 4xL40S hybrid configuration:")
print("   - VLM: GPUs 2,3")
print("   - Remote LLM API configured")
print("\n📋 Current .env relevant settings:")
!grep -E "NVIDIA_VISIBLE_DEVICES|USE_REMOTE_LLM|REMOTE_LLM" .env

Before proceeding, make sure you have compose.yaml, config.yaml and .env in the current directory

In [ ]:
!ls -a

#### Update docker compose version (recommended v2.32.4)

In [ ]:
!docker compose version

!mkdir -p ~/.docker/cli-plugins
!curl -SL https://github.com/docker/compose/releases/latest/download/docker-compose-linux-x86_64 -o ~/.docker/cli-plugins/docker-compose
!chmod +x ~/.docker/cli-plugins/docker-compose

In [ ]:
!docker compose version

### Step 7: Launch the Blueprint with 4xL40S Hybrid Configuration

Here, we start the docker container to spin up the blueprint. The container performs:
1. Downloads the VLM for GPUs 2,3
2. Performs model calibration
3. Generates a TRT-LLM Engine for VLM
4. Spins up the Milvus and Neo4J database
5. Starts Video Search and Summarization service with hybrid config
6. Finally, we get the frontend and backend endpoints

<div class="alert alert-block alert-success">
    <b>Note:</b>  This step takes around 20-25 minutes for first run (faster than 8-GPU setup since no LLM download!)
</div>

The below code cell filters out important logs so that the output is not very verbose

In [ ]:
%%capture
!docker compose down

In [ ]:
import subprocess
import time

keywords = ["Milvus server started", "Downloading model", "Downloaded model", "VILA Embeddings", "VILA TRT model load execution time", 
            "Starting quantization", "Quantization done", "Engine generation completed", "TRT engines generated", "Uvicorn", 
            "VIA Server loaded", "Backend", "Frontend", "****", "Remote LLM", "Hybrid"]

# Start the docker compose process in detached mode
subprocess.run(['docker', 'compose', 'up', '--quiet-pull', '-d'])

def filter_logs(logs, keywords):
    return [line for line in logs.splitlines() if any(keyword in line for keyword in keywords)]

printed_lines = set()

print("🚀 Starting 4xL40S Hybrid VSS Deployment...")
print("💡 Configuration: Local VLM(GPUs 2,3) + Reranker(GPU 0) + Embedding(GPU 1) + Remote LLM")
print("⏱️  Expected time: 20-25 minutes (faster than 8-GPU setup!)\n")

try:
    while True:
        logs = subprocess.check_output(['docker', 'compose', 'logs', '--no-color'], universal_newlines=True)
        filtered_logs = filter_logs(logs, keywords)
        new_logs = [line for line in filtered_logs if line not in printed_lines]
        
        for line in new_logs:
            print(line)
            printed_lines.add(line)
            if "Frontend" in line:
                print("\n🎉 VSS 4xL40S Hybrid Server ran successfully!")
                print("💰 Cost optimization: 50% savings compared to 8xL40S")
                print("🔗 Access VSS Frontend UI from Brev portal tunnels section. Refer to Step 8 for details.")
                raise SystemExit
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping log tailing...")
except SystemExit:
    pass

### Step 8: Access VSS UI with Brev Tunnels

1. Go to the "Access" Tab on Brev and scroll down to the "Using Tunnels" section   
    <img src="images/brev_access_tab.png" alt="Access Tab" width="1200"/>

2. (Optional) Add port "9100". This is set in .env file which is configurable   
    <img src="images/brev_add_port.png" alt="Access Tab" width="1200"/>

3. Click on the frontend port (9100) Sharable URL link to access blueprint UI  
    <div class="alert alert-block alert-success">
        <b>Note:</b>  Please reload the brev page if the shareable URL is showing as unhealthy or follow the alternative steps with VSCode below.
    </div>
    <img src="images/brev_vss_ui_url.png" alt="Access Tab" width="1200"/>

4. Experience VSS using the gradio based UI application with your 4xL40S hybrid setup! 
   
   **🎯 Your Setup Performance:**
   - Video analysis: Excellent (2 GPUs for VLM)
   - Text processing: Fast (local Reranker & Embedding)
   - Question answering: High-quality (remote LLM)
   - Cost: 50% savings vs 8-GPU setup
   
   For quick steps to summarize a video, refer to [this link](https://docs.nvidia.com/vss/latest/content/sample_summarization.html). Additionally, for detailed instructions on how to use the UI, follow [this guide](https://docs.nvidia.com/vss/latest/content/ui_app.html)  
    <img src="images/vss_landing_page.png" alt="Access Tab" width="1200"/>

### [Alternative Option] Access instance on VSCode and add ports

1. First, go to the "Access" tab  
    <img src="images/brev_access_tab.png" alt="Access Tab" width="1200"/>

2. Refer to the commands in "Using Brev CLI" to open VSCode locally  
    <div class="alert alert-block alert-success">
        <b>Note:</b>  Make sure to <a href="https://code.visualstudio.com/docs/setup/mac#_configure-the-path-with-vs-code" target="_blank">Configure the path with VS Code</a> if you run into erros while accessing instance through VSCode.
    </div>  
    <img src="images/brev_vscode_access.png" alt="Access Tab" width="1200"/>

3. Navigate to the Ports view in the Panel region (Ports: Focus on Ports View), and select Forward a Port  
    <img src="images/vscode_ports.png" alt="VSCode Ports" width="1200"/>

4. Add frontend (9100) and backend (8100) ports, and access blueprint UI by clicking on the "localhost:9100"  
    <img src="images/vscode_access_vss_ui.png" alt="VSCode Ports" width="1200"/>

### 📊 4xL40S Hybrid Setup Summary

**🎯 Your Configuration:**
- **GPU 0**: Reranker (Local) - Fast text ranking
- **GPU 1**: Embedding (Local) - Quick vector search  
- **GPU 2,3**: VLM (Local) - Excellent video analysis
- **Remote**: LLM - High-quality responses

**💰 Cost Benefits:**
- 50% reduction in GPU costs
- Faster deployment (no LLM download)
- Same video analysis quality

**⚡ Performance:**
- Video understanding: Excellent (2 dedicated GPUs)
- Search & ranking: Fast (local models)
- Q&A responses: High-quality (remote LLM)

Congratulations on your optimized video search and summarization setup!

### Uninstalling the blueprint

Uncomment the following cell to stop the blueprint container, followed by stopping all model containers.

In [ ]:
# %%capture
# # To bring down the blueprint instance
# !docker compose down

# # To stop all other containers (Reranker and Embedding)
# !docker stop $(docker ps -q)

# print("🛑 All VSS components stopped")
# print("💡 Your 4xL40S hybrid configuration is saved and ready to restart anytime!")

In [ ]:
!docker ps
print("\n📊 Expected containers for 4xL40S setup:")
print("   • VSS main container (blueprint)")
print("   • Reranker NIM (GPU 0)")
print("   • Embedding NIM (GPU 1)")
print("   • Database containers (Milvus, Neo4j)")
print("\n🚫 NOT running locally: LLM (using remote API)")